In [ ]:
import os
import sys
import urllib.request
import json
import time
import openai  
from utils import generate_text_GPT

# openai 키, 모델
openai.api_key = "sk-"
gpt_model = "gpt-3.5-turbo"#"gpt-4"#"gpt-3.5-turbo" #gpt-4-0314

# 네이버 파파고 번역 서비스
# => https://developers.naver.com/docs/papago/papago-nmt-overview.md
client_id = "" # 개발자센터에서 발급받은 Client ID 값
client_secret = ""       # 개발자센터에서 발급받은 Client Secret 값

PROMPT_CONTEXT = '내용에서 질문에 대한 답을 찾아 답변해주세요.\n답이 없으면 내용을 요약해 답변해주세요.\n답변은 간략히 100자 이내로 답변해주세요.\n질문:{query}\n답변:{context}'

#PROMPT_CONTEXT = 'Please find the answer to the Query in the Contents and answer it.\nIf there is no answer, please summarize the Contents.\nPlease answer briefly within 100 characters.\nQuery:{query}\nContents:{context}'

query:str = "1박2일 출장시, 숙박비는 얼마?"

context:str = """
6. 일비, 숙박비 및 식비
6.1. 일비, 숙박비, 식비의 지급
6.1.1. 국내 여행자의 일비, 숙박비 및 식비는 아래에 따라 지급한다. 
다만, 부득이한 사유로 숙박비의 상한을 초과하는 여비 지출이 필요한 경우, 국내 여행의 경우에는 숙박비 상한액의 10분의 3을 넘지 아니하는 범위에서 여비를 추가로 지급할 수 있다. 이 경우, 국외 여행의 경우, 대표이사의 사전 승인을 득해야 한다.
6.1.2. 일비는 여행일수에 따라 지급하되, 법인차량을 이용하는 경우에는 일비의 2분의 1을 지급한다.
6.1.3. 식비는 여행일수에 따라 지급한다. 다만, 수로여행과 항공여행에 는 따로 식비가 필요한 경우에만 식비를 지급한다.
 1) 부장 이하는 1일비는 20,000원, 식비는 20,000원 지원하고, 1일 숙박비는 1박당 실비로 지원하는데, 단 서울특별시는 사항액이 70,000원, 광역시는 60,000원, 그 밖의 지역은 50,000원 임
 2) 임원 이상: 1일비는 20,000원, 1일 숙박비는 1박당 실비로 지원하며,1일 식비는 25,000원 지원함.
 
6.1.1. 국외 여행자의 일비, 숙박비 및 식비는 아래에 따라 지급한다. 
다만, 부득이한 사유로 숙박비의 상한을 초과하는 여비 지출이 필요한 경우, 국외 여행의 경우 2분의 1을 넘지 아니하는 범위에서 여비를 추가로 지급할 수 있다. 이 경우,국외 여행의 경우, 대표이사의 사전 승인을 득해야 한다.
6.1.2. 일비는 여행일수에 따라 지급하되, 법인차량을 이용하는 경우에는 일비의 2분의 1을 지급한다.
6.1.3. 식비는 여행일수에 따라 지급한다. 다만, 수로여행과 항공여행에 는 따로 식비가 필요한 경우에만 식비를 지급한다.
-전무 이상 임원은 일비 40달러이고 숙박비는 실비로 계산하며 상한액은 282 달러이고 식비는 282달러임.
-임원은 일비 35달러이고 숙박비는 실비로 계산하며 상한액은 223 달러이고 식비는 107달러임.
-부장은 일비 30달러이고 숙박비는 실비로 계산하며 상한액은 176 달러이고 식비는 81달러임.

7.2. 이전비의 지급
7.2.1. 이전비 지급은 아래에 따라  지급한다.
  -국내 이전비: 5톤 이하이 이사물인 경우에는 이전비의 실비 지급(사다리차 이용로 포함)
  -국내 이전비: 5톤을 초과하는 이사화물(이사화물이 7.5톤을 넘는 경우 7.5톤을 상한으로 한다)인 경우에는, 5톤의 이사화물에 해다하는 이전비의 실비에 5톤 초과 7.5톤 이하의 이사화물에 해당하는 이전비의 실비(사다리차 이용료 포함)의 50%를 더한 금액
  -국외 이전비 : 15세제곱미터 이하의 이사화물은 실비 지급
  -국외 이전비 : 5세제곱미터를 넘는 이사화물(이사화물이 25세제곱미터를 넘는 경우에는 25세제곱미터를 상한으로 한다)인 경우, 15세제곱미터의 이사화물에 해당하는 이전비 실비에 15세제곱미터 초과 25세제곱미터 이하의 이사화물에 해당하는 이전비 실비의 50퍼센트를 더한 금액
7.2.2. 이전한 날의 다음날부터 기산하여 6개월 이내에 거주지 변경 및 이사화물의 운송명세(이동구간, 이동거리, 운송비 등을 말한다)를 확인할 수 있는 증빙서류(세금계산서, invoice 등)를 갖추어 이전비를 신청할 수 있다. 
다만, 회사를 계약 주체로 하여 세금계산서 등을 발행할 수 있는 업체와 거래하는 경우, 회사가 직접 지급할 수 있다.
"""

query_in = PROMPT_CONTEXT.format(query=query, context=context)
print(f'query_in:{query_in}')

In [ ]:
# 질문에 대해 한국어 -> 영어로 번역
def translate(query:str, source_lang:str, target_lang:str, 
              client_id:str, client_secret:str):
   
    out_query:str = ""
    start_time = time.time()

    encText = urllib.parse.quote(query)

    
    #data = "source=ko&target=en&text=" + encText
    data = f"source={source_lang}&target={target_lang}&text={encText}"
    print(f'data:{data}')
    url = "https://openapi.naver.com/v1/papago/n2mt"

    request = urllib.request.Request(url)
    request.add_header("X-Naver-Client-Id",client_id)
    request.add_header("X-Naver-Client-Secret",client_secret)

    response = urllib.request.urlopen(request, data=data.encode("utf-8"))
    rescode = response.getcode()

    if(rescode==200):
        response_body = response.read()
        response_body.decode('utf-8')

        # JSON 문자열을 딕셔너리로 변환
        my_response = json.loads(response_body)
        out_query = my_response["message"]["result"]["translatedText"].strip("'")
        print(out_query)

    else:
        print("Error Code:" + rescode)

     # 소요된 시간을 계산합니다.
    end_time = time.time()
    formatted_elapsed_time = "{:.2f}".format(end_time - start_time)
    print(f'time:{formatted_elapsed_time}')
    
    return out_query
        

In [ ]:
query = translate(query=query_in, source_lang="ko", target_lang="en", 
        client_id=client_id, client_secret=client_secret);

#query += context
#print(query)

In [ ]:
#system_prompt = "Please answer like a counselor."
system_prompt = ""
prompt = query
answer1 = generate_text_GPT(gpt_model=gpt_model, prompt=prompt, system_prompt=system_prompt)
print(answer1)

In [ ]:
# 영어->한국어로 번역
answer = translate(query=answer1, source_lang="en", target_lang="ko", 
         client_id=client_id, client_secret=client_secret);

print(answer)

In [ ]:
#system_prompt = "상담원 처럼 답변해주세요."
system_prompt = ""
prompt = query_in
answer1 = generate_text_GPT(gpt_model=gpt_model, prompt=prompt, system_prompt=system_prompt)
print(answer1)